In [12]:
# ensure we run on the CPU
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"
import netket as nk
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Couplings J1 and J2
J = [1, 0.2]
L = 6

In [3]:
# Define custom graph
edge_colors = []
for i in range(L):
    edge_colors.append([i, (i+1)%L, 1])
    edge_colors.append([i, (i+2)%L, 2])
# Define the netket graph object
g = nk.graph.Graph(edges=edge_colors)

In [4]:
#Sigma^z*Sigma^z interactions
sigmaz = [[1, 0], [0, -1]]
mszsz = (np.kron(sigmaz, sigmaz))
#Exchange interactions
exchange = np.asarray([[0, 0, 0, 0], [0, 0, 2, 0], [0, 2, 0, 0], [0, 0, 0, 0]])
bond_operator = [
    (J[0] * mszsz).tolist(),
    (J[1] * mszsz).tolist(),
    (-J[0] * exchange).tolist(),  
    (J[1] * exchange).tolist(),
]
bond_color = [1, 2, 1, 2]

In [5]:
# Spin based Hilbert Space
hi = nk.hilbert.Spin(s=0.5, total_sz=0.0, N=g.n_nodes)

In [6]:
# Custom Hamiltonian operator
op = nk.operator.GraphOperator(hi, graph=g, bond_ops=bond_operator, bond_ops_colors=bond_color)

In [7]:
import netket.nn as nknn
import flax.linen as nn
import jax.numpy as jnp

class FFNN(nn.Module):
    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=2*x.shape[-1], 
                     use_bias=True, 
                     param_dtype=np.complex128, 
                     kernel_init=nn.initializers.normal(stddev=0.01), 
                     bias_init=nn.initializers.normal(stddev=0.01)
                    )(x)
        x = nknn.log_cosh(x)
        x = jnp.sum(x, axis=-1)
        return x
        
model = FFNN()

In [8]:
# We shall use an exchange Sampler which preserves the global magnetization (as this is a conserved quantity in the model)
sa = nk.sampler.MetropolisExchange(hilbert=hi, graph=g, d_max = 2)
# Construct the variational state
vs = nk.vqs.MCState(sa, model, n_samples=1008)

In [11]:
vs.parameters

{'Dense_0': {'bias': Array([ 0.00238289+3.74842818e-03j,  0.00010437-5.61405965e-03j,
          0.00189761+2.51057832e-05j, -0.00488054-2.06314300e-03j,
          0.00971112-8.85560764e-04j, -0.01109759+2.36959943e-03j,
          0.01209808-1.46258066e-02j, -0.00435821-9.66881267e-04j,
         -0.00443665-1.69935942e-04j,  0.00588698-3.00515425e-03j,
          0.00480799-2.13545279e-03j, -0.00190509-5.47481396e-03j],      dtype=complex128),
  'kernel': Array([[ 3.68010283e-03-0.01482067j,  4.26662848e-03-0.00057493j,
          -2.02035395e-03-0.00337572j, -7.23701743e-03-0.00532163j,
          -1.04652802e-02+0.00311879j,  5.93203786e-03+0.00050563j,
          -1.64054773e-03+0.00460847j, -1.21088253e-02+0.01167829j,
           3.18955093e-03-0.00466516j,  4.04826654e-03+0.001041j  ,
           7.39663719e-03-0.00043449j,  6.32389037e-04+0.00090199j],
         [ 6.69586781e-03-0.00691999j,  3.44914615e-03+0.00829999j,
          -3.75922210e-05+0.00827567j, -1.43251032e-02-0.0008529j ,

In [10]:
# We choose a basic, albeit important, Optimizer: the Stochastic Gradient Descent
opt = nk.optimizer.Sgd(learning_rate=0.01)
# Stochastic Reconfiguration
sr = nk.optimizer.SR(diag_shift=0.01)
# We can then specify a Variational Monte Carlo object, using the Hamiltonian, sampler and optimizers chosen.
# Note that we also specify the method to learn the parameters of the wave-function: here we choose the efficient
# Stochastic reconfiguration (Sr), here in an iterative setup
gs = nk.VMC(hamiltonian=op, optimizer=opt, variational_state=vs, preconditioner=sr)

In [9]:
# We need to specify the local operators as a matrix acting on a local Hilbert space 
sf = []
sites = []
structure_factor = nk.operator.LocalOperator(hi, dtype=complex)
for i in range(0, L):
    for j in range(0, L):
        structure_factor += (nk.operator.spin.sigmaz(hi, i)*nk.operator.spin.sigmaz(hi, j))*((-1)**(i-j))/L

In [16]:
gs.run(out='test', n_iter=10, obs={'Structure Factor': structure_factor})

100%|█| 10/10 [00:00<00:00, 32.41it/s, Energy=-10.248148-0.000002j ± 0.000081 [σ


(JsonLog('test', mode=write, autoflush_cost=0.005)
   Runtime cost:
   	Log:    0.0018014907836914062
   	Params: 0.0003819465637207031,)

In [17]:
vs.parameters

{'Dense_0': {'bias': Array([-1.81629820e-03+2.15431465e-03j,  9.06613752e-04-2.06271448e-04j,
          2.26035195e-04-3.28127170e-04j,  2.08335038e-03-2.25164023e-03j,
          1.64110843e-03-7.19567654e-04j, -8.67396718e-03+4.81214492e-03j,
          7.67041687e-03-1.05888738e-02j, -7.28421177e-04-1.24842268e-03j,
         -6.53813152e-05+7.72731380e-05j,  3.48209240e-03-6.46399307e-03j,
          6.88732086e-03-1.55875674e-03j,  5.05305920e-03-7.16233863e-03j],      dtype=complex128),
  'kernel': Array([[-1.15940586e-01-0.15782856j,  3.74926536e-03+0.27062573j,
          -7.84357394e-04-0.18712648j,  6.22721004e-02+0.10524423j,
          -7.78622589e-02+0.08524305j,  3.82310284e-02+0.04001182j,
          -4.86812889e-02+0.09033862j, -8.06919108e-02+0.09981888j,
          -4.37392998e-02-0.04148786j,  4.11363585e-02-0.1643214j ,
           1.37978281e-02-0.04387173j,  3.20269923e-02-0.05832318j],
         [ 1.01647604e-01-0.23017303j, -7.36842099e-03+0.15160045j,
          -1.543744

In [14]:
# Run the optimization protocol
gs.run(out='test', n_iter=100, obs={'Structure Factor': structure_factor})

100%|█| 100/100 [00:03<00:00, 30.84it/s, Energy=-10.248297+0.000011j ± 0.000065 


(JsonLog('test', mode=write, autoflush_cost=0.005)
   Runtime cost:
   	Log:    0.013495445251464844
   	Params: 0.0007162094116210938,)

In [15]:
vs.parameters

{'Dense_0': {'bias': Array([-1.83210421e-03+2.21330318e-03j,  9.98089519e-04-1.24758424e-04j,
          2.55253480e-04-3.16717775e-04j,  2.01696104e-03-2.23207451e-03j,
          1.65989143e-03-6.79093888e-04j, -8.78853811e-03+4.73349355e-03j,
          7.66758232e-03-1.07556233e-02j, -7.87963896e-04-1.28473825e-03j,
          1.64357990e-05+4.40169974e-05j,  3.51866248e-03-6.52227172e-03j,
          6.88216903e-03-1.56973272e-03j,  5.03095672e-03-7.10760143e-03j],      dtype=complex128),
  'kernel': Array([[-1.15942507e-01-0.15782194j,  3.75488580e-03+0.27062706j,
          -7.93523989e-04-0.18713515j,  6.22725019e-02+0.10524353j,
          -7.78621613e-02+0.08524187j,  3.82386294e-02+0.04001828j,
          -4.86823218e-02+0.09032735j, -8.06953955e-02+0.09981246j,
          -4.37393799e-02-0.0414867j ,  4.11366058e-02-0.16432929j,
           1.37924777e-02-0.04387462j,  3.20215735e-02-0.05833126j],
         [ 1.01641905e-01-0.23017108j, -7.36218355e-03+0.15160498j,
          -1.541314

In [18]:
gs.run(out='test', n_iter=200, obs={'Structure Factor': structure_factor})

100%|█| 200/200 [00:06<00:00, 29.39it/s, Energy=-10.248163+0.000017j ± 0.000030 


(JsonLog('test', mode=write, autoflush_cost=0.005)
   Runtime cost:
   	Log:    0.028252363204956055
   	Params: 0.0011048316955566406,)

In [19]:
vs.parameters

{'Dense_0': {'bias': Array([-1.15643220e-03+0.00115498j,  3.23577795e-04-0.00075728j,
          2.47689912e-04-0.00012379j,  2.77973411e-03-0.00276988j,
          1.45187892e-03-0.00151618j, -7.03270771e-03+0.00607504j,
          7.56375549e-03-0.00818141j, -6.49076007e-05-0.0003078j ,
         -1.42492094e-03+0.00054368j,  3.01381388e-03-0.0053264j ,
          6.96799042e-03-0.00125694j,  5.46356458e-03-0.00786817j],      dtype=complex128),
  'kernel': Array([[-1.15923400e-01-0.15788817j,  3.76358574e-03+0.27066585j,
          -7.62010588e-04-0.18711433j,  6.22879363e-02+0.10526923j,
          -7.78427813e-02+0.08526248j,  3.81334288e-02+0.0399023j ,
          -4.86069215e-02+0.09052637j, -8.06683947e-02+0.09987521j,
          -4.37602614e-02-0.04150075j,  4.11053427e-02-0.16421222j,
           1.38807195e-02-0.04382005j,  3.20786034e-02-0.05819891j],
         [ 1.01659526e-01-0.23017918j, -7.36197221e-03+0.15161897j,
          -1.55688263e-03+0.13722901j, -1.15446759e-01+0.01874955j,

In [20]:
gs.run(out='test', n_iter=400, obs={'Structure Factor': structure_factor})

100%|█| 400/400 [00:13<00:00, 28.64it/s, Energy=-10.248191-0.000010j ± 0.000017 


(JsonLog('test', mode=write, autoflush_cost=0.005)
   Runtime cost:
   	Log:    0.05749654769897461
   	Params: 0.002638101577758789,)

In [21]:
vs.parameters

{'Dense_0': {'bias': Array([-0.00073591+5.34630301e-04j,  0.00010945-6.69134270e-04j,
          0.00022558+1.72846369e-05j,  0.00303376-3.35287054e-03j,
          0.00161766-1.87542674e-03j, -0.00594754+6.86697407e-03j,
          0.00717592-6.76069189e-03j,  0.00020875+1.80425324e-04j,
         -0.00206854+1.08430456e-03j,  0.00271379-4.23145487e-03j,
          0.00707016-9.82474413e-04j,  0.00578867-7.93143887e-03j],      dtype=complex128),
  'kernel': Array([[-1.15925520e-01-0.15790904j,  3.76569370e-03+0.27067999j,
          -7.40055536e-04-0.18712721j,  6.23154096e-02+0.10525812j,
          -7.78370848e-02+0.08525163j,  3.80760202e-02+0.03982948j,
          -4.85232557e-02+0.0906483j , -8.06538363e-02+0.09988106j,
          -4.37759889e-02-0.04151162j,  4.10858114e-02-0.16412287j,
           1.39379765e-02-0.04378435j,  3.21156083e-02-0.05812046j],
         [ 1.01663391e-01-0.23016006j, -7.36313518e-03+0.15161654j,
          -1.56488896e-03+0.1372395j , -1.15450530e-01+0.01879169j,